In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import requests as req
from splinter import Browser
from selenium import webdriver
import time

In [18]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [28]:
years = [2015, 2016, 2017, 2018, 2019]
tables = []

In [29]:
for year in years:
    url =f"https://www.nfl.com/standings/league/{year}/reg/"
    
    browser.visit(url)
    time.sleep(5)
    table = pd.read_html(url)
    tables.append(table)
    df = table 
    


AttributeError: 'list' object has no attribute 'to_csv'

IndexError: list index out of range